In [1]:
%pip install -qU datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install sentence-transformers

# Data Loading

In [3]:
from datasets import load_dataset

dataset_restaurant = load_dataset("jakartaresearch/semeval-absa", name='restaurant')
dataset_laptop = load_dataset("jakartaresearch/semeval-absa", name='laptop')
dataset_fabsa = load_dataset("jordiclive/FABSA")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

semeval-absa.py:   0%|          | 0.00/5.49k [00:00<?, ?B/s]

restaurant/train/0000.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

restaurant/validation/0000.parquet:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/105k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7930 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1057 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1587 [00:00<?, ? examples/s]

# Data Preprocessing

## Expand Dataset into a convenient format

In [4]:
from tqdm.auto import tqdm
from typing import TypedDict, Generator
from datasets import Dataset, Features, Value, ClassLabel, DatasetDict


class LabeledExample(TypedDict):
    aspect: str
    labels: str
    sentence: str


def expand_dataset_semeval(dataset: Dataset) -> Generator[LabeledExample, None, None]:
    POLARITY_SET = {'positive', 'negative', 'neutral'}
    for data in tqdm(dataset):
        sentence = data['text']  # type: ignore
        terms = data['aspects']['term']  # type: ignore
        polarities = data['aspects']['polarity']  # type: ignore

        for term, polarity in zip(terms, polarities):
            if polarity not in POLARITY_SET or term == '':
                continue
            yield {
                'aspect': term,
                'labels': polarity,
                'sentence': sentence
            }


def get_dataset_semeval(dataset: DatasetDict) -> DatasetDict:
    ds_features = Features({
        'aspect': Value('string'),
        'labels': ClassLabel(names=['positive', 'negative', 'neutral']),
        'sentence': Value('string')
    })

    ds = DatasetDict(
        {
            split_name: \
            Dataset.from_generator(
                expand_dataset_semeval,
                features=ds_features,
                gen_kwargs={'dataset': dataset[split_name]}
            )
            for split_name in ['train', 'validation']
        }
    )
    return ds


def expand_dataset_fabsa(dataset: Dataset) -> Generator[LabeledExample, None, None]:
    POLARITY_SET = {'positive', 'negative', 'neutral'}
    for data in tqdm(dataset):
        sentence = data['text']  # type: ignore

        for term, polarity in data['labels']:
            if polarity not in POLARITY_SET or term == '':
                continue
            yield {
                'aspect': term.split(": ")[-1],
                'labels': polarity,
                'sentence': sentence
            }


def get_dataset_fabsa(dataset: DatasetDict) -> DatasetDict:
    ds_features = Features({
        'aspect': Value('string'),
        'labels': ClassLabel(names=['positive', 'negative', 'neutral']),
        'sentence': Value('string')
    })

    ds = DatasetDict(
        {
            split_name: \
            Dataset.from_generator(
                expand_dataset_fabsa,
                features=ds_features,
                gen_kwargs={'dataset': dataset[split_name]}
            )
            for split_name in ['train', 'validation', 'test']
        }
    )
    return ds


## Tokenize

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "microsoft/deberta-v3-base" # 'google-bert/bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)  # Must be like BertTokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Load and preprocess datasets into raw (non-tokenized) format
ds_restaurant_raw = get_dataset_semeval(dataset_restaurant)
ds_laptop_raw = get_dataset_semeval(dataset_laptop)
ds_fabsa_raw = get_dataset_fabsa(dataset_fabsa)

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/3044 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/800 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

  0%|          | 0/7930 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/1057 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/1587 [00:00<?, ?it/s]

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm  # for progress bar
import numpy as np

# Load a pre-trained sentence transformer model for augmentation
augmentation_model = SentenceTransformer('all-mpnet-base-v2')  # Best for semantic similarity

def augment_dataset(dataset, model):
    augmented_data = []
    for example in tqdm(dataset):
        sentence = example['sentence']
        aspect = example['aspect']

        # Compute embeddings
        sentence_embedding = model.encode(sentence)
        aspect_embedding = model.encode(aspect)

        # Compute cosine similarity and take its absolute value (modulus)
        similarity = abs(cosine_similarity([sentence_embedding], [aspect_embedding])[0][0])

        # If the aspect is unrelated (mod of similarity < threshold), replace the label with neutral
        if similarity < 0.0001:  # Adjust threshold as needed
            example['labels'] = 2  # Replace label with neutral

        # Add the example to the augmented dataset
        augmented_data.append({
            'aspect': aspect,
            'labels': example['labels'],
            'sentence': sentence
        })

    return Dataset.from_list(augmented_data)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Augment all datasets
augmented_restaurant_train = augment_dataset(ds_restaurant_raw['train'], augmentation_model)

100%|██████████| 3608/3608 [01:41<00:00, 35.70it/s]


In [9]:
augmented_laptop_train = augment_dataset(ds_laptop_raw['train'], augmentation_model)


100%|██████████| 2328/2328 [01:04<00:00, 36.23it/s]


In [10]:
augmented_fabsa_train = augment_dataset(ds_fabsa_raw['train'], augmentation_model)

100%|██████████| 13998/13998 [06:32<00:00, 35.67it/s]


In [11]:
from datasets import concatenate_datasets
# Combine augmented training datasets
train_dataset_raw = concatenate_datasets([
    augmented_restaurant_train,
    augmented_laptop_train,
    augmented_fabsa_train
])

In [12]:
def tokenize_function(examples):
    """This tokenizer returns the format -
    [CLS] aspect tokens [SEP] sentence tokens [SEP]
    """
    return tokenizer(examples['aspect'], examples['sentence'], padding=True, truncation=True, return_token_type_ids=False, max_length=512)

In [13]:
# Tokenize the combined augmented training dataset
train_dataset = train_dataset_raw.map(tokenize_function, batched=True)

Map:   0%|          | 0/19934 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [14]:
ds_restaurant_val = ds_restaurant_raw['validation'].map(tokenize_function, batched=True)
ds_laptop_val = ds_laptop_raw['validation'].map(tokenize_function, batched=True)
ds_fabsa_test = ds_fabsa_raw['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

Map:   0%|          | 0/2812 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


# Training

In [15]:
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import concatenate_datasets


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    # per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    # logging_steps=10,
    # evaluation_strategy="epoch",     # evaluation is done at the end of each epoch
    resume_from_checkpoint=False,
    learning_rate=5e-5,
    report_to="none"
)

# # Freeze all the parameters in the model
# for param in model.deberta.parameters():
#     param.requires_grad = False

# # Enable gradients for the classifier layer (last layer)
# for param in model.classifier.parameters():
#     param.requires_grad = True

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch import nn
from sklearn.metrics import precision_recall_fscore_support



CLASS_WEIGHTS = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_dataset['labels']),
    y=train_dataset['labels']
)

In [16]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')

        # Define fixed weights for positive, negative, and neutral classes
        fixed_weights = torch.tensor([1.0, 1.0, 2]).to(logits.device).float()

        # Compute custom loss with fixed weights
        loss_fct = nn.CrossEntropyLoss(weight=fixed_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


In [17]:
trainer = CustomTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,
    data_collator=DataCollatorWithPadding(tokenizer, padding=True, max_length=512)
)

In [18]:
model.train()
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
500,0.655900
1000,0.520200
1500,0.470300
2000,0.424500
2500,0.397500
3000,0.315700
3500,0.331700
4000,0.296900
4500,0.311700
5000,0.289300


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

TrainOutput(global_step=7476, training_loss=0.32487087239550677, metrics={'train_runtime': 7883.4913, 'train_samples_per_second': 7.586, 'train_steps_per_second': 0.948, 'total_flos': 1.512708511278744e+16, 'train_loss': 0.32487087239550677, 'epoch': 3.0})

# Evaluation

In [19]:
import evaluate
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)  # Ensure predictions are derived correctly

    # Load the f1 metric from evaluate
    f1 = evaluate.load('f1')

    # Calculate F1 score using the 'macro' average method
    f1_result = f1.compute(predictions=predictions, references=labels, average='macro')

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(labels, predictions)

    # Return a dictionary with the metric name and its value
    return {"f1_score": f1_result['f1'], "confusion_matrix": conf_matrix, 'classification_report': classification_report(labels, predictions)}

In [20]:
model.eval()
pred = trainer.predict(ds_restaurant_val)
restaurant_metrics = compute_metrics(
    (pred.predictions, ds_restaurant_val['labels']))
print(restaurant_metrics['classification_report'])

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.92      0.98      0.94       728
           1       0.84      0.89      0.87       196
           2       0.82      0.58      0.68       196

    accuracy                           0.89      1120
   macro avg       0.86      0.81      0.83      1120
weighted avg       0.89      0.89      0.88      1120



In [21]:
print("\nConfusion Matrix:")
print(restaurant_metrics['confusion_matrix'])


Confusion Matrix:
[[710   7  11]
 [  8 174  14]
 [ 57  25 114]]


```
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       728
           1       0.85      0.85      0.85       196
           2       0.80      0.59      0.68       196

    accuracy                           0.88      1120
   macro avg       0.85      0.80      0.82      1120
weighted avg       0.88      0.88      0.88      1120
```

In [22]:
model.eval()
pred = trainer.predict(ds_laptop_val)

laptop_metrics = compute_metrics(
    (pred.predictions, ds_laptop_val['labels']))
print(laptop_metrics['classification_report'])

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.90      0.91      0.91       341
           1       0.64      0.93      0.76       128
           2       0.81      0.50      0.62       169

    accuracy                           0.81       638
   macro avg       0.78      0.78      0.76       638
weighted avg       0.82      0.81      0.80       638



In [23]:
print("\nConfusion Matrix:")
print(laptop_metrics['confusion_matrix'])


Confusion Matrix:
[[311  17  13]
 [  2 119   7]
 [ 33  51  85]]


```
              precision    recall  f1-score   support

           0       0.92      0.90      0.91       341
           1       0.72      0.91      0.80       128
           2       0.77      0.64      0.70       169

    accuracy                           0.83       638
   macro avg       0.80      0.82      0.80       638
weighted avg       0.84      0.83      0.83       638
```

In [24]:
model.eval()
pred = trainer.predict(ds_fabsa_test)

fabsa_metrics = compute_metrics(
    (pred.predictions, ds_fabsa_test['labels']))
print(fabsa_metrics['classification_report'])

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1825
           1       0.94      0.91      0.93       896
           2       0.83      0.81      0.82        91

    accuracy                           0.95      2812
   macro avg       0.91      0.90      0.91      2812
weighted avg       0.95      0.95      0.95      2812



In [25]:
print("\nConfusion Matrix:")
print(fabsa_metrics['confusion_matrix'])


Confusion Matrix:
[[1780   39    6]
 [  69  818    9]
 [   6   11   74]]


```
              precision    recall  f1-score   support

           0       0.95      0.94      0.94      1825
           1       0.92      0.79      0.85       896
           2       0.31      0.81      0.44        91

    accuracy                           0.89      2812
   macro avg       0.73      0.85      0.75      2812
weighted avg       0.92      0.89      0.90      2812
```

In [26]:
# from huggingface_hub import notebook_login

# notebook_login()

In [27]:
# model.push_to_hub('deberta-v3-base-absa-semeval2014-fabsa', private=True)

In [28]:
# tokenizer.push_to_hub('deberta-v3-base-absa-semeval2014-fabsa', private=True)